In [120]:
# Import libraries and dependencies
import os
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import datetime 

%matplotlib inline

In [121]:
# Load .env enviroment variables
from dotenv import load_dotenv
load_dotenv()

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create an object for aplpaca api 
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)


In [122]:
sectors_csv = Path("Resources/spy_etf_sectors.csv")
sectors_df = pd.read_csv(sectors_csv)

sectors_df = sectors_df.drop(columns=["Zacks\nG Sector", "\nCompany", "% of\nPortfolio"])

sectors_df = sectors_df.rename(columns={
    "\nTicker": "ticker",
    "Russell\nSector": "sector"
})
sectors_df

,ticker,sector
0,ATVI,TECHNOLOGY
1,GOOGL,TECHNOLOGY
2,GOOG,TECHNOLOGY
3,T,UTILITIES
4,CTL,UTILITIES
...,...,...
499,PEG,UTILITIES
500,SRE,UTILITIES
501,SO,UTILITIES
502,WEC,UTILITIES


In [123]:
dxy_csv = Path("Resources/$dxy_historical_data_01-01_to_09-15-2020.csv")
dxy_df = pd.read_csv(dxy_csv)
dxy_df = dxy_df.rename(columns={"Last":"DXY", "Time":"date"})
dxy_df = dxy_df[["date", "DXY"]].dropna()

In [124]:
btc_csv = Path("Resources/btcusd_historical_data_01-01_to_09-15-2020.csv")
btc_df = pd.read_csv(btc_csv)
btc_df = btc_df.rename(columns={"Last":"BTC", "Time":"date"}).dropna()
btc_df = btc_df[["date", "BTC"]]

In [125]:
xau_csv = Path("Resources/btcusd_historical_data_01-01_to_09-15-2020.csv")
xau_df = pd.read_csv(xau_csv)
xau_df = xau_df.rename(columns={"Last":"XAU", "Time":"date"}).dropna()
xau_df = xau_df[["date", "XAU"]]

In [132]:
alternative_assets = xau_df.merge(btc_df, on='date')
alternative_assets = alternative_assets.merge(dxy_df, on='date')
alternative_assets['date'] = pd.to_datetime(alternative_assets['date'], utc=True).dt.date
alternative_assets

,date,XAU,BTC,DXY
0,2020-09-15,10866.25,10866.25,93.07
1,2020-09-14,10692.27,10692.27,93.05
2,2020-09-11,10330.04,10330.04,93.27
3,2020-09-10,10304.28,10304.28,93.34
4,2020-09-09,10260.84,10260.84,93.26
...,...,...,...,...
179,2020-01-08,8003.06,8003.06,97.31
180,2020-01-07,8018.82,8018.82,96.96
181,2020-01-06,7580.60,7580.60,96.62
182,2020-01-03,7278.13,7278.13,96.89


In [127]:
# Set the ticker to stripes of 100
ticker_list = sectors_df["ticker"].to_list()
ticker_stripe_1 = ticker_list[0:99]
ticker_stripe_2 = ticker_list[99:198]
ticker_stripe_3 = ticker_list[198:297]
ticker_stripe_4 = ticker_list[297:396]
ticker_stripe_5 = ticker_list[396:495]
ticker_stripe_6 = ticker_list[495:]

# Set timeframe to '1D'
timeframe = "1D"

# Set start and end datetimes of from Jan 1 2020 to Sep 15 2020
start_date = start = pd.Timestamp("2020-01-01", tz="America/Chicago").isoformat()
end_date = pd.Timestamp("2020-09-15", tz="America/Chicago").isoformat()

# Chunk the data by stripes to make it compaitible with alpaca api ingestion 

spy_stripe_1 = api.get_barset(
    ticker_stripe_1,
    timeframe,
    start=start_date,
    end=end_date
).df

spy_stripe_2 = api.get_barset(
    ticker_stripe_2,
    timeframe,
    start=start_date,
    end=end_date
).df

spy_stripe_3 = api.get_barset(
    ticker_stripe_3,
    timeframe,
    start=start_date,
    end=end_date
).df

spy_stripe_4 = api.get_barset(
    ticker_stripe_4,
    timeframe,
    start=start_date,
    end=end_date
).df

spy_stripe_5 = api.get_barset(
    ticker_stripe_5,
    timeframe,
    start=start_date,
    end=end_date
).df

spy_stripe_6 = api.get_barset(
    ticker_stripe_6,
    timeframe,
    start=start_date,
    end=end_date
).df

# Combine all of the stripes together 
spy_df = pd.concat([spy_stripe_1, spy_stripe_2, spy_stripe_3, spy_stripe_4, spy_stripe_5, spy_stripe_6], axis=1)
spy_df

AAP                                        ADM  \
                             open     high       low   close   volume   open   
2020-01-02 00:00:00-05:00  160.47  160.950  157.5300  159.40   726650  46.57   
2020-01-03 00:00:00-05:00  158.03  159.630  157.2500  159.48   482371  45.89   
2020-01-06 00:00:00-05:00  157.74  158.890  156.4325  156.82   571689  45.79   
2020-01-07 00:00:00-05:00  156.90  157.400  152.4090  154.95  1021104  45.57   
2020-01-08 00:00:00-05:00  154.85  156.050  153.1400  153.15   706120  45.11   
...                           ...      ...       ...     ...      ...    ...   
2020-09-09 00:00:00-04:00  152.62  155.680  152.6200  154.30  1040694  45.51   
2020-09-10 00:00:00-04:00  155.53  155.550  153.1050  153.57   650622  46.47   
2020-09-11 00:00:00-04:00  154.47  155.320  153.0800  154.56   879257  46.13   
2020-09-14 00:00:00-04:00  156.90  158.915  156.1300  157.35  1322800  47.00   
2020-09-15 00:00:00-04:00  157.89  158.410  156.5100  156.89   415068  47.78   

                                                            ...     WEC  \
                             high      low  close   volume  ...    open   
2020-01-02 00:00:00-05:00  46.640  45.8800  46.11  1899517  ...  92.400   
2020-01-03 00:00:00-05:00  46.190  45.7200  46.02  1512616  ...  90.650   
2020-01-06 00:00:00-05:00  45.845  45.3200  45.67  2045946  ...  91.340   
2020-01-07 00:00:00-05:00  45.690  45.1000  45.11  1567481  ...  90.890   
2020-01-08 00:00:00-05:00  45.190  44.3800  44.61  2640257  ...  91.210   
...                           ...      ...    ...      ...  ...     ...   
2020-09-09 00:00:00-04:00  46.340  45.4700  46.06  5600146  ...  96.300   
2020-09-10 00:00:00-04:00  46.590  45.9386  46.02  4053865  ...  97.275   
2020-09-11 00:00:00-04:00  46.970  46.0100  46.89  5603942  ...  96.110   
2020-09-14 00:00:00-04:00  47.760  46.9600  47.50  5657532  ...  96.550   
2020-09-15 00:00:00-04:00  47.900  47.2700  47.47  2027878  ...  97.750   

                                                                  XEL  \
                              high     low  close     volume     open   
2020-01-02 00:00:00-05:00  92.5200  90.455  90.68  1267002.0  63.5500   
2020-01-03 00:00:00-05:00  91.7100  90.520  91.22  1405515.0  62.2800   
2020-01-06 00:00:00-05:00  91.8000  90.800  91.34   907752.0  62.7600   
2020-01-07 00:00:00-05:00  91.0700  90.340  90.96   930917.0  62.1500   
2020-01-08 00:00:00-05:00  91.3850  90.510  91.11   806742.0  62.5900   
...                            ...     ...    ...        ...      ...   
2020-09-09 00:00:00-04:00  99.0400  95.970  97.76  1872756.0  69.1400   
2020-09-10 00:00:00-04:00  97.6700  95.860  95.96  1707418.0  70.2400   
2020-09-11 00:00:00-04:00  96.3000  94.930  95.92  1619722.0  69.2500   
2020-09-14 00:00:00-04:00  97.7717  95.830  97.14  1169398.0  68.4268   
2020-09-15 00:00:00-04:00  99.2800  97.750  99.18   866768.0  69.8600   

                                                              
                              high      low   close   volume  
2020-01-02 00:00:00-05:00  63.5800  62.2000  62.380  2340833  
2020-01-03 00:00:00-05:00  62.9150  62.2800  62.680  1908930  
2020-01-06 00:00:00-05:00  62.9400  62.3300  62.590  1392103  
2020-01-07 00:00:00-05:00  62.5168  61.9700  62.460  1620406  
2020-01-08 00:00:00-05:00  62.6400  62.1200  62.420  1630637  
...                            ...      ...     ...      ...  
2020-09-09 00:00:00-04:00  71.3500  69.1400  70.590  1662544  
2020-09-10 00:00:00-04:00  70.6750  69.0400  69.140  1650643  
2020-09-11 00:00:00-04:00  69.3500  68.1600  68.849  1865467  
2020-09-14 00:00:00-04:00  69.9000  68.3495  69.540  1408925  
2020-09-15 00:00:00-04:00  71.1500  69.8150  70.410  1606302  

[178 rows x 2520 columns]

In [128]:
spy_df.to_csv(r'C:\Users\achri\OneDrive\Desktop\FinTech\Homework\stock_indy_500\spy_df_csv.csv', index = True)
spy_df_csv = Path(r'C:\Users\achri\OneDrive\Desktop\FinTech\Homework\stock_indy_500\spy_df_csv.csv')
spy_csv_df = pd.read_csv(spy_df_csv, header=None )
spy_csv_df[0] = pd.to_datetime(spy_csv_df[0], utc=True).dt.date
spy_csv_df

,0,1,2,3,4,5,6,7,8,9,...,2511,2512,2513,2514,2515,2516,2517,2518,2519,2520
0,NaT,AAP,AAP,AAP,AAP,AAP,ADM,ADM,ADM,ADM,...,WEC,WEC,WEC,WEC,WEC,XEL,XEL,XEL,XEL,XEL
1,NaT,open,high,low,close,volume,open,high,low,close,...,open,high,low,close,volume,open,high,low,close,volume
2,2020-01-02,160.47,160.95,157.53,159.4,726650,46.57,46.64,45.88,46.11,...,92.4,92.52,90.455,90.68,1267002.0,63.55,63.58,62.2,62.38,2340833
3,2020-01-03,158.03,159.63,157.25,159.48,482371,45.89,46.19,45.72,46.02,...,90.65,91.71,90.52,91.22,1405515.0,62.28,62.915,62.28,62.68,1908930
4,2020-01-06,157.74,158.89,156.4325,156.82,571689,45.79,45.845,45.32,45.67,...,91.34,91.8,90.8,91.34,907752.0,62.76,62.94,62.33,62.59,1392103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,2020-09-09,152.62,155.68,152.62,154.3,1040694,45.51,46.34,45.47,46.06,...,96.3,99.04,95.97,97.76,1872756.0,69.14,71.35,69.14,70.59,1662544
176,2020-09-10,155.53,155.55,153.105,153.57,650622,46.47,46.59,45.9386,46.02,...,97.275,97.67,95.86,95.96,1707418.0,70.24,70.675,69.04,69.14,1650643
177,2020-09-11,154.47,155.32,153.08,154.56,879257,46.13,46.97,46.01,46.89,...,96.11,96.3,94.93,95.92,1619722.0,69.25,69.35,68.16,68.849,1865467
178,2020-09-14,156.9,158.915,156.13,157.35,1322800,47.0,47.76,46.96,47.5,...,96.55,97.7717,95.83,97.14,1169398.0,68.4268,69.9,68.3495,69.54,1408925


In [129]:
#new_header = spy_csv_df.iloc[0] #grab the first row for the header
#spy_csv_df = spy_csv_df[1:] #take the data less the header row
#spy_csv_df.columns = new_header #set the header row as the df header

header_row= 1
spy_csv_df.columns = spy_csv_df.iloc[header_row]
spy_csv_df.drop(['open', 'high', 'low', 'volume'], axis=1, inplace=True)
spy_csv_df.columns = spy_csv_df.iloc[0]

In [133]:
spy_csv_df= spy_csv_df[2:]
spy_csv_df.columns.values[0]='date'
spy_csv_df

final_df = spy_csv_df.merge(alternative_assets, on='date',)
final_df

,date,AAP,ADM,AMZN,APTV,ATVI,AZO,BBY,BKNG,BWA,...,PEG,PNW,PPL,SO,SRE,WEC,XEL,XAU,BTC,DXY
0,2020-01-08,153.15,44.61,1892.09,93.09,59.87,1147.22,88.6,2062.54,42.7,...,57.95,88.52,35.39,62.62,149.89,91.11,62.42,8003.06,8003.06,97.31
1,2020-01-09,152.88,44.15,1901.0,91.75,59.34,1146.29,89.37,2085.43,41.94,...,58.4,89.89,35.5,63.26,148.87,91.99,62.54,7788.86,7788.86,97.41
2,2020-01-10,149.0,43.945,1882.98,89.49,58.95,1132.09,90.65,2086.9,41.41,...,58.3,90.57,35.44,63.665,149.76,92.54,62.64,8049.33,8049.33,97.34
3,2020-01-13,145.29,44.43,1891.41,88.86,59.24,1128.6,91.25,2080.05,41.38,...,58.25,91.26,35.44,64.54,151.17,93.67,63.36,8133.67,8133.67,97.37
4,2020-01-14,148.94,44.49,1869.2,88.99,60.05,1136.6,90.46,2071.71,41.7,...,58.36,91.99,35.58,64.92,151.21,93.95,63.47,8733.99,8733.99,97.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,2020-09-09,154.3,46.06,3268.27,85.61,79.58,1232.38,108.85,1872.84,42.29,...,53.44,72.63,27.74,52.85,120.96,97.76,70.59,10260.84,10260.84,93.26
170,2020-09-10,153.57,46.02,3176.12,84.09,79.1,1217.64,107.37,1819.12,41.67,...,52.29,71.39,27.28,51.55,118.09,95.96,69.14,10304.28,10304.28,93.34
171,2020-09-11,154.56,46.89,3117.28,84.53,77.95,1233.53,107.15,1781.23,42.37,...,52.4,70.9,27.53,51.76,117.8,95.92,68.849,10330.04,10330.04,93.27
172,2020-09-14,157.35,47.5,3102.97,85.9,79.13,1241.91,107.69,1782.635,42.81,...,52.48,72.07,27.91,52.87,119.12,97.14,69.54,10692.27,10692.27,93.05


,0,1,2,3,4,5,6,7,8,9,...,164,165,166,167,168,169,170,171,172,173
date,2020-01-08,2020-01-09,2020-01-10,2020-01-13,2020-01-14,2020-01-15,2020-01-16,2020-01-17,2020-01-21,2020-01-22,...,2020-09-01,2020-09-02,2020-09-03,2020-09-04,2020-09-08,2020-09-09,2020-09-10,2020-09-11,2020-09-14,2020-09-15
AAP,153.15,152.88,149.0,145.29,148.94,150.23,151.94,151.05,149.01,147.64,...,156.39,158.22,155.19,153.79,151.73,154.3,153.57,154.56,157.35,156.89
ADM,44.61,44.15,43.945,44.43,44.49,44.4,45.34,45.24,45.12,44.93,...,45.15,45.74,45.81,46.5,45.22,46.06,46.02,46.89,47.5,47.47
AMZN,1892.09,1901.0,1882.98,1891.41,1869.2,1862.01,1877.95,1864.71,1892.55,1887.46,...,3500.47,3529.34,3369.17,3294.62,3149.84,3268.27,3176.12,3117.28,3102.97,3153.28
APTV,93.09,91.75,89.49,88.86,88.99,87.89,91.22,91.52,91.02,91.11,...,87.04,89.99,86.76,86.23,84.67,85.61,84.09,84.53,85.9,85.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WEC,91.11,91.99,92.54,93.67,93.95,95.17,96.17,96.4,97.69,98.23,...,93.59,97.41,96.36,96.46,95.61,97.76,95.96,95.92,97.14,99.18
XEL,62.42,62.54,62.64,63.36,63.47,64.47,65.03,65.36,66.0,66.22,...,68.55,71.54,70.66,70.02,69.11,70.59,69.14,68.849,69.54,70.41
XAU,8003.06,7788.86,8049.33,8133.67,8733.99,8820.93,8682.42,8906.77,8725.8,8640.85,...,12020.7,11374.5,10785.9,10604.4,10015.8,10260.8,10304.3,10330,10692.3,10866.2
BTC,8003.06,7788.86,8049.33,8133.67,8733.99,8820.93,8682.42,8906.77,8725.8,8640.85,...,12020.7,11374.5,10785.9,10604.4,10015.8,10260.8,10304.3,10330,10692.3,10866.2
